# TNT Cates: Flow Curve Fitting
> **Handbook:** This notebook demonstrates the TNT Cates model. For complete mathematical derivations and theoretical background, see [TNT Cates Documentation](../../docs/source/models/tnt/tnt_cates.rst).


**Estimated Time:** 3-5 minutes

## Welcome to TNT Cates (Living Polymers / Wormlike Micelles)

This notebook is the **gateway** to the TNT Cates model — a molecular theory for **living polymers** and **wormlike micelles** where chains can reversibly **break and recombine** through scission-recombination kinetics.

### What are Living Polymers?

Living polymers are supramolecular assemblies where the **chain length is not fixed** but fluctuates through reversible bond breaking/forming:

- **Wormlike micelles**: Surfactant self-assemblies that undergo reversible scission (CTAB, CPCl, EHAC)
- **Telechelic polymers** with reversible end-group association
- **Supramolecular polymers** with dynamic bonding along the backbone

The key difference from TNT Single-Mode: **chain breaking** (not just crosslink breaking) introduces a **second timescale** — the chain scission/recombination time $\tau_{break}$.

### Cates Model Physics

The Cates model combines:

1. **Reptation dynamics**: Chains diffuse through a tube with reptation time $\tau_{rep} \sim L^3$ (L = chain contour length)
2. **Reversible scission**: Chains break with rate $1/\tau_{break}$ and recombine instantly
3. **Geometric mean relaxation**: The effective timescale emerges from the interplay: $\tau_d = \sqrt{\tau_{rep} \cdot \tau_{break}}$

This $\tau_d$ is the **dominant relaxation time** observed in experiments — it's **independent of chain length** (a unique Cates signature).

### Key Signatures

| Feature | Cates Prediction | Physical Origin |
|---------|------------------|-----------------|
| **Flow curve** | Shear thinning with $\eta \sim \dot{\gamma}^{-2/3}$ | Stress-induced scission or orientation |
| **SAOS** | Cole-Cole semicircle, single $\tau_d$ | Geometric mean balances reptation + scission |
| **Relaxation** | Bi-exponential decay (fast breaking + slow reptation) | Two-timescale kinetics |
| **Shear banding** | Flow instability at intermediate $\dot{\gamma}$ | Mechanical/constitutional coupling |

### Cates vs. Other TNT Models

| Model | Chain Length | Relaxation Mechanism | Shear Response |
|-------|--------------|----------------------|----------------|
| **SingleMode** | Fixed | Crosslink breakage only | No shear thinning (constant $\beta$) |
| **Cates** | Fluctuating | Reptation + scission | Shear thinning, shear banding |
| **Loop-Bridge** | Fixed | Loop/bridge conversion | Shear thickening then thinning |
| **Sticky Rouse** | Fixed | Multi-mode sticker dynamics | Hierarchical relaxation |

### Mathematical Framework

The Cates model extends the TNT conformation tensor evolution with:

$$
\frac{d\mathbf{S}}{dt} = \mathbf{L} \cdot \mathbf{S} + \mathbf{S} \cdot \mathbf{L}^T + g_0 \mathbf{I} - \beta_{Cates}(\mathbf{S}) \mathbf{S}
$$

where the breakage rate $\beta_{Cates}$ depends on both:
- **Reptation**: $\beta_{rep} \sim 1/\tau_{rep}(L)$
- **Scission**: $\beta_{break} \sim 1/\tau_{break}$

The **effective relaxation rate** is:

$$
\beta_{eff} = \frac{1}{\tau_d}, \quad \tau_d = \sqrt{\tau_{rep} \cdot \tau_{break}}
$$

> **Cates Model Handbook**  
> For detailed physics, equations, and validation studies, see:  
> [../../docs/source/models/tnt/tnt_cates.rst](../../docs/source/models/tnt/tnt_cates.rst)

### Six Protocols in This Series

Each Cates notebook explores a different rheological test:

1. **FLOW_CURVE** (NB07, this notebook): Shear thinning and shear banding
2. **STARTUP** (NB08): Stress overshoot with two-timescale relaxation
3. **RELAXATION** (NB09): Bi-exponential decay ($\tau_{break}$ fast, $\tau_{rep}$ slow)
4. **CREEP** (NB10): Delayed flow with stress-dependent yielding
5. **SAOS** (NB11): Cole-Cole semicircle, single $\tau_d$ from two timescales
6. **LAOS** (NB12): Nonlinear harmonics and flow instabilities

> **TNT Protocols Reference**  
> For the mathematical framework shared by all TNT models, see:  
> [../../docs/source/models/tnt/tnt_protocols.rst](../../docs/source/models/tnt/tnt_protocols.rst)

### Recommended Learning Path

**New to Cates?** Follow this sequence:
1. NB07 (this notebook): Flow curve shear thinning
2. NB11: SAOS Cole-Cole plot (single $\tau_d$ from two timescales)
3. NB09: Bi-exponential relaxation
4. NB08: Startup overshoot

**Advanced topics**:
- NB12: LAOS nonlinearity and shear banding
- NB10: Creep with stress-dependent yielding

> **TNT Knowledge Extraction Guide**  
> For interpreting fitted parameters (extracting $\tau_{rep}$, $\tau_{break}$ from $\tau_d$ and other observables), see:  
> [../../docs/source/models/tnt/tnt_knowledge_extraction.rst](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)

---

## Learning Objectives (This Notebook)

1. Understand living polymer physics and reversible scission kinetics
2. Fit TNT Cates model to flow curve data
3. Extract geometric mean relaxation time $\tau_d = \sqrt{\tau_{rep} \tau_{break}}$
4. Predict shear thinning from stress-induced chain breaking
5. Analyze shear banding instabilities
6. Perform Bayesian inference for parameter uncertainty

## Prerequisites

- Understanding of TNT fundamentals (Notebook 01)
- Familiarity with reptation theory (optional but helpful)
- Knowledge of shear banding phenomena (for advanced analysis)

## Runtime Estimate

- NLSQ fitting: ~5-10 seconds
- Bayesian inference (demo): ~60-120 seconds
- Total: ~5-8 minutes

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import matplotlib.pyplot as plt
import numpy as np

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64

verify_float64()

from rheojax.models.tnt import TNTCates

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    compute_cates_tau_d,
    compute_fit_quality,
    get_tnt_cates_param_names,
    load_ml_ikh_flow_curve,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
)
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

param_names = get_tnt_cates_param_names()
print(f"TNTCates parameters: {param_names}")

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## Theory: Cates Model for Wormlike Micelles

The Cates model describes living polymers (wormlike micelles) undergoing reversible scission and recombination:

**Key time scales:**
- $\tau_{\text{rep}}$: Reptation time (chain diffusion)
- $\tau_{\text{break}}$: Breaking time (scission rate)
- $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$: Effective relaxation time

**Flow curve (steady shear):**
$$\sigma(\dot{\gamma}) = \frac{G_0 \tau_d \dot{\gamma}}{1 + (\tau_d \dot{\gamma})^2} + \eta_s \dot{\gamma}$$

**Fast-breaking limit:** When $\zeta = \tau_{\text{break}}/\tau_{\text{rep}} \ll 1$, the model reduces to single-mode Maxwell with $\tau = \tau_d$.

**Non-monotonic flow curve:** If $\partial\sigma/\partial\dot{\gamma} < 0$, shear banding may occur.

## Load Flow Curve Data

In [ ]:
gamma_dot, stress = load_ml_ikh_flow_curve("ARES_up")

print(f"Data points: {len(gamma_dot)}")
print(f"Shear rate range: {gamma_dot.min():.2e} to {gamma_dot.max():.2e} s^-1")
print(f"Stress range: {stress.min():.2f} to {stress.max():.2f} Pa")

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(gamma_dot, stress, 'o', label='Experimental data', markersize=6)
ax.set_xlabel(r'Shear rate $\dot{\gamma}$ (s$^{-1}$)', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Flow Curve Data', fontsize=14)
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTCates()

start_time = time.time()
model.fit(gamma_dot, stress, test_mode='flow_curve', method='scipy')
fit_time = time.time() - start_time

print(f"\nNLSQ Optimization completed in {fit_time:.2f} seconds")

# Extract fitted parameters
nlsq_params = {name: model.parameters.get_value(name) for name in param_names}
print("\nNLSQ Parameters:")
for name, value in nlsq_params.items():
    print(f"  {name}: {value:.4e}")

# Compute fit quality
stress_pred_fit = model.predict(gamma_dot, test_mode='flow_curve')
quality = compute_fit_quality(stress, stress_pred_fit)
print(f"\nFit Quality: R² = {quality['R2']:.6f}, RMSE = {quality['RMSE']:.4e}")

## Visualize NLSQ Fit

In [ ]:
# Compute metrics for plot title
metrics = compute_fit_quality(stress, model.predict(gamma_dot, test_mode='flow_curve'))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    gamma_dot, stress, model, test_mode="flow_curve",
    param_names=param_names, log_scale=True,
    xlabel=r'Shear rate $\dot{\gamma}$ (s$^{-1}$)',
    ylabel=r'Shear stress $\sigma$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})'
)
plt.close("all")

## Physical Analysis

In [ ]:
# Generate fine grid for smooth predictions
gamma_dot_pred = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)
stress_pred = model.predict(gamma_dot_pred, test_mode='flow_curve')

tau_d = compute_cates_tau_d(nlsq_params['tau_rep'], nlsq_params['tau_break'])
zeta = nlsq_params['tau_break'] / nlsq_params['tau_rep']

print(f"\nPhysical Analysis:")
print(f"  Reptation time (tau_rep): {nlsq_params['tau_rep']:.4e} s")
print(f"  Breaking time (tau_break): {nlsq_params['tau_break']:.4e} s")
print(f"  Effective relaxation time (tau_d): {tau_d:.4e} s")
print(f"  Fast-breaking parameter (zeta): {zeta:.4f}")

if zeta < 0.1:
    print(f"  → Fast-breaking limit (zeta < 0.1): Model approximates single-mode Maxwell")
else:
    print(f"  → Not in fast-breaking limit: Full Cates dynamics active")

d_stress = jnp.gradient(stress_pred, gamma_dot_pred)
if jnp.any(d_stress < 0):
    print(f"  ⚠ Non-monotonic flow curve detected: Potential shear banding")
else:
    print(f"  ✓ Monotonic flow curve: No shear banding predicted")

## Bayesian Inference with NUTS

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        gamma_dot, stress,
        test_mode='flow_curve',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
posterior = bayesian_result.posterior_samples

bayesian_params = {name: float(jnp.mean(posterior[name])) for name in param_names}
param_std = {name: float(jnp.std(posterior[name])) for name in param_names}

print("\nPosterior Statistics:")
for name in param_names:
    print(f"  {name}: {bayesian_params[name]:.4e} ± {param_std[name]:.4e}")

# Compare NLSQ vs Bayesian using the utility function
print_parameter_comparison(model, posterior, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayesian_result, 'to_inference_data'):
    display_arviz_diagnostics(bayesian_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive Distribution

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayesian_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        gamma_dot,
        stress,
        model, bayesian_result, test_mode="flow_curve",
        param_names=param_names, log_scale=True,
        xlabel=r'Shear rate $\\dot{\\gamma}$ (s$^{-1}$)',
        ylabel=r'Shear stress $\\sigma$ (Pa)'
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

## Physical Interpretation from Posterior

In [ ]:
tau_d_posterior = np.sqrt(posterior['tau_rep'] * posterior['tau_break'])
zeta_posterior = posterior['tau_break'] / posterior['tau_rep']

print(f"\nPhysical quantities from posterior:")
print(f"  tau_d: {np.mean(tau_d_posterior):.4e} ± {np.std(tau_d_posterior):.4e} s")
print(f"  zeta (tau_break/tau_rep): {np.mean(zeta_posterior):.4f} ± {np.std(zeta_posterior):.4f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(tau_d_posterior, bins=30, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tau_d_posterior), color='r', linestyle='--', label='Mean')
ax1.set_xlabel(r'$\tau_d$ (s)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.set_title('Effective Relaxation Time Distribution', fontsize=12)

ax2.hist(zeta_posterior, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(zeta_posterior), color='r', linestyle='--', label='Mean')
ax2.axvline(0.1, color='g', linestyle=':', label='Fast-breaking limit')
ax2.set_xlabel(r'$\zeta = \tau_{break}/\tau_{rep}$', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.set_title('Fast-Breaking Parameter Distribution', fontsize=12)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Save Results

In [ ]:
save_tnt_results(model, bayesian_result, "cates", "flow_curve", param_names)

## Key Takeaways

1. **TNTCates captures living polymer dynamics** through reptation and scission times
2. **Effective relaxation time** $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$ governs flow curve shape
3. **Fast-breaking limit** ($\zeta \ll 1$) reduces to single-mode Maxwell behavior
4. **Non-monotonic flow curves** indicate potential shear banding in wormlike micelles
5. **Bayesian inference** quantifies uncertainty in time scales and plateau modulus

**Next steps:** Explore other protocols (startup, SAOS, LAOS) to validate extracted parameters across different deformation modes.

## Next Steps

- **Notebook 08**: Continue exploring this model family
- **Advanced Models**: Compare with other TNT variants (Notebooks 07-30)


## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.


### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
